# 1-2

In [1]:
using JuMP, HiGHS

# Roaster data: a, b, c for each roaster 1-6
a = [20, 5, 4, 20, 10, 15]
b = [2, 3, 5, 3, 1, 4]
c = [10, 15, 20, 5, 25, 15]

E = zeros(6, 6)  # 6 roasters × 6 weeks (columns 1-6 represent weeks 0-5)
for i in 1:6
    for t in 0:5
        E[i, t+1] = b[i] * (a[i] + t)^2 - c[i] * (a[i] + t)
    end
end

model = Model(HiGHS.Optimizer)

@variable(model, x[1:6, 1:6] >= 0) 
@constraint(model, [i=1:6], sum(x[i, t] for t in 1:6) == 1)
@constraint(model, [t=1:6], sum(x[i, t] for i in 1:6) == 1)
@objective(model, Max, sum(E[i, t] * x[i, t] for i in 1:6, t in 1:6))

set_silent(model)
optimize!(model)

In [4]:
println("Total Enjoyment: ", objective_value(model))
for i in 1:6
    for t in 1:6
        if value(x[i, t]) == 1
            println("Roaster $i → Week $(t-1) (Enjoyment = $(E[i, t]))")
        end
    end
end

Total Enjoyment: 3665.0
Roaster 1 → Week 3 (Enjoyment = 828.0)
Roaster 2 → Week 1 (Enjoyment = 18.0)
Roaster 3 → Week 2 (Enjoyment = 60.0)
Roaster 4 → Week 5 (Enjoyment = 1750.0)
Roaster 5 → Week 0 (Enjoyment = -150.0)
Roaster 6 → Week 4 (Enjoyment = 1159.0)


# 2-6

In [6]:
model = Model(HiGHS.Optimizer)
@variable(model, y1 >= 0)
@variable(model, y2 >=0)
@constraint(model, 4y1 + 5y2 >= 3)
@constraint(model, y1 + 2y2 >= 1)
@objective(model, Min, 7y1 + 12y2)
set_silent(model)
optimize!(model)
println("Optimal value of y1: ", value(y1))
println("Optimal value of y2: ", value(y2))
println("Objective value: ", objective_value(model))

Optimal value of y1: 0.33333333333333337
Optimal value of y2: 0.3333333333333333
Objective value: 6.333333333333334


# 2-7

In [7]:
model = Model(HiGHS.Optimizer)
@variable(model, y1 >= 0)
@variable(model, y2 >=0)
@constraint(model, 4y1 + 5y2 >= 3)
@constraint(model, y1 + 2y2 >= 1)
@objective(model, Min, 7y1 + 15y2)
set_silent(model)
optimize!(model)
println("Optimal value of y1: ", value(y1))
println("Optimal value of y2: ", value(y2))
println("Objective value: ", objective_value(model))

Optimal value of y1: 1.0
Optimal value of y2: 0.0
Objective value: 7.0


# 3-2

In [8]:
using JuMP
using GLPK

# Define the set of nodes (restaurants)
nodes = ["S", "G", "O", "R", "I", "W", "A"]  

arcs = [
    ("S", "G"),
    ("S", "O"),
    ("S", "R"),
    ("G", "O"),
    ("G", "I"),
    ("O", "R"),
    ("O", "I"),
    ("O", "W"),
    ("R", "W"),
    ("I", "W"),
    ("I", "A"),
    ("W", "A")
]

# Define the cost for each arc (in minutes)
cost = Dict(
    ("S", "G") => 4.0,
    ("S", "O") => 6.0,
    ("S", "R") => 5.0,
    ("G", "O") => 1.0,
    ("G", "I") => 7.0,
    ("O", "R") => 2.0,
    ("O", "I") => 5.5,
    ("O", "W") => 4.0,
    ("R", "W") => 5.0,
    ("I", "W") => 1.0,
    ("I", "A") => 6.0,
    ("W", "A") => 8.0
)

model_dual = Model(GLPK.Optimizer)
@variable(model_dual, u[node in nodes])

# For each arc (i, j) in A, add the constraint: u[i] - u[j] <= cost[i,j]
for (i, j) in arcs
    @constraint(model_dual, u[i] - u[j] <= cost[(i,j)])
end

@objective(model_dual, Max, u["S"] - u["A"])
optimize!(model_dual)
println("Optimal dual objective value: ", objective_value(model_dual))
println("Optimal node potentials (u):")
for node in nodes
    println("u[$node] = ", value(u[node]))
end


Optimal dual objective value: 16.5
Optimal node potentials (u):
u[S] = 5.0
u[G] = 1.0
u[O] = 0.0
u[R] = 0.0
u[I] = -5.5
u[W] = -3.5
u[A] = -11.5


# 4-1

In [11]:
using JuMP
using GLPK

source = "Store"
sink   = "Customer"
# the 7 wand types
types = ["T1", "T2", "T3", "T4", "T5", "T6", "T7"]
owls = ["O1", "O2", "O3", "O4", "O5"]

all_nodes = [source; types; owls; sink]
arcs = Vector{Tuple{String,String}}()
cap = Dict{Tuple{String,String},Float64}()

for t in types
    push!(arcs, (source, t))
    cap[(source, t)] = length(types) # each owl carries at most 1 wand of each type
end

for t in types
    for o in owls
        push!(arcs, (t, o))
        cap[(t, o)] = 1.0
    end
end

owl_caps = Dict("O1" => 6.0, "O2" => 5.0, "O3" => 4.0, "O4" => 2.0, "O5" => 10.0)
for o in owls
    push!(arcs, (o, sink))
    cap[(o, sink)] = owl_caps[o]
end

model = Model(GLPK.Optimizer)
@variable(model, x[arc in arcs] >= 0)
@variable(model, f >= 0)

# Flow conservation constraints:
# For the source: outflow - inflow = f
@constraint(model, sum(x[(source, j)] for (i, j) in arcs if i == source) 
                  - sum(x[(i, source)] for (i, j) in arcs if j == source) == f)

# For the sink: inflow - outflow = f
@constraint(model, sum(x[(i, sink)] for (i, j) in arcs if j == sink)
                  - sum(x[(sink, j)] for (i, j) in arcs if i == sink) == f)

# For all other nodes, net flow must be 0.
for n in setdiff(all_nodes, [source, sink])
    @constraint(model, sum(x[(i, j)] for (i, j) in arcs if i == n)
                     - sum(x[(i, j)] for (i, j) in arcs if j == n) == 0)
end

for arc in arcs
    @constraint(model, x[arc] <= cap[arc])
end

@objective(model, Max, f)
optimize!(model)

println("Optimal total number of wands delivered (max flow): ", objective_value(model))
println("\nWands carried by each owl:")
for o in owls
    # Flow on arc (owl, sink) gives the number of wands carried by that owl.
    println("$o carries ", value(x[(o, sink)]), " wands.")
end

println("\nNumber of each type of wand shipped:")
for t in types
    # Flow on arc (Store, t) gives the number of wands of that type shipped.
    println("$t: ", value(x[(source, t)]))
end


Optimal total number of wands delivered (max flow): 24.0

Wands carried by each owl:
O1 carries 6.0 wands.
O2 carries 5.0 wands.
O3 carries 4.0 wands.
O4 carries 2.0 wands.
O5 carries 7.0 wands.

Number of each type of wand shipped:
T1: 5.0
T2: 5.0
T3: 4.0
T4: 4.0
T5: 3.0
T6: 2.0
T7: 1.0
